In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy.random as rnd
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import scipy as sp
from scipy.stats import norm
import pandas_datareader as pd_data
from pandas_datareader import data as pdr
import yfinance as yf
from datetime import datetime
from statsmodels.tsa.stattools import acf, pacf
from scipy.interpolate import interp1d
from numpy import arange, array, exp
import Inputs

In [2]:
SURFACE_PATH = "data/swaption_atm_vol_full.xlsx"
DATA_PATH = "data/forward_sofr_swap_full.xlsx"

In [3]:
swaption_df = pd.read_excel(SURFACE_PATH)
swaption_df

,TERM (TENOR),1M,1M.1,1M.2,1M.3,1M.4,1M.5,1M.6,1M.7,1M.8,...,25Y.8,30Y,30Y.1,30Y.2,30Y.3,30Y.4,30Y.5,30Y.6,30Y.7,30Y.8
0,MATRUITY (EXPIRY),1Y,2Y,3Y,4Y,5Y,6Y,7Y,8Y,9Y,...,9Y,1Y,2Y,3Y,4Y,5Y,6Y,7Y,8Y,9Y
1,Ticker,USSNAA1 ICPL Curncy,USSNAA2 ICPL Curncy,USSNAA3 ICPL Curncy,USSNAA4 ICPL Curncy,USSNAA5 ICPL Curncy,USSNAA6 ICPL Curncy,USSNAA7 ICPL Curncy,USSNAA8 ICPL Curncy,USSNAA9 ICPL Curncy,...,USSNA259 ICPL Curncy,USSNA301 ICPL Curncy,USSNA302 ICPL Curncy,USSNA303 ICPL Curncy,USSNA304 ICPL Curncy,USSNA305 ICPL Curncy,USSNA306 ICPL Curncy,USSNA307 ICPL Curncy,USSNA308 ICPL Curncy,USSNA309 ICPL Curncy
2,2025-01-10 00:00:00,72.2,96.8,97.7,99.9,101.6,100.5,99.5,99.1,98.6,...,70.3,73.9,75,74.2,73.2,72.1,71.1,69.8,68.6,67.1
3,2025-01-09 00:00:00,79.9,101.1,102.2,103.3,104.5,103.2,101.8,101.3,100.8,...,69.7,73.1,74.2,73.5,72.4,71.4,70.4,69.1,67.9,66.4
4,2025-01-08 00:00:00,80.9,101.7,102.9,104,105.2,103.9,102.5,102,101.5,...,69.8,73.2,74.3,73.6,72.6,71.5,70.5,69.2,68,66.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,2023-01-09 00:00:00,109.8,133.9,131,127.7,124.9,123,121.1,120.4,119.7,...,55.2,59.1,57.4,56.7,56,55.3,54.7,54.1,53.2,52.3
505,2023-01-06 00:00:00,105.7,129.8,126.3,123.6,121.3,119.4,117.4,117.2,117,...,55.6,59.7,58.1,57.3,56.7,56,55.3,54.7,53.6,52.6
506,2023-01-05 00:00:00,108.2,132.2,130.8,129.6,129,127.2,125.4,125,124.5,...,56,60.1,58.4,57.7,57,56.3,55.6,55,54,53
507,2023-01-04 00:00:00,111.6,136.4,134.8,133,131.5,130.4,129.3,128.8,128.4,...,56.4,60.6,58.9,58.1,57.4,56.6,56,55.3,54.3,53.3


In [4]:
forwards_df = pd.read_excel(DATA_PATH)
forwards_df

,TERM (TENOR),1M,1M.1,1M.2,1M.3,1M.4,1M.5,1M.6,1M.7,1M.8,...,25Y.8,30Y,30Y.1,30Y.2,30Y.3,30Y.4,30Y.5,30Y.6,30Y.7,30Y.8
0,MATRUITY (EXPIRY),1Y,2Y,3Y,4Y,5Y,6Y,7Y,8Y,9Y,...,9Y,1Y,2Y,3Y,4Y,5Y,6Y,7Y,8Y,9Y
1,Ticker,S0042FS 1Y1M BLC Curncy,S0042FS 2Y1M BLC Curncy,S0042FS 3Y1M BLC Curncy,S0042FS 4Y1M BLC Curncy,S0042FS 5Y1M BLC Curncy,S0042FS 6Y1M BLC Curncy,S0042FS 7Y1M BLC Curncy,S0042FS 8Y1M BLC Curncy,S0042FS 9Y1M BLC Curncy,...,S0042FS 9Y25Y BLC Curncy,S0042FS 1Y30Y BLC Curncy,S0042FS 2Y30Y BLC Curncy,S0042FS 3Y30Y BLC Curncy,S0042FS 4Y30Y BLC Curncy,S0042FS 5Y30Y BLC Curncy,S0042FS 6Y30Y BLC Curncy,S0042FS 7Y30Y BLC Curncy,S0042FS 8Y30Y BLC Curncy,S0042FS 9Y30Y BLC Curncy
2,2025-01-10 00:00:00,4.0661,4.1379,4.1364,4.1339,4.1491,4.1527,4.1593,4.1726,4.1962,...,3.7727,4.0118,3.9763,3.9358,3.8934,3.8497,3.8027,3.7535,3.7021,3.6474
3,2025-01-09 00:00:00,3.9148,3.9922,4.0277,4.0657,4.1016,4.1192,4.1407,4.1716,4.2065,...,3.8087,4.0041,3.978,3.9465,3.9107,3.8715,3.8282,3.7818,3.732,3.6784
4,2025-01-08 00:00:00,3.9364,4.0082,4.0314,4.0687,4.097,4.1133,4.1532,4.1709,4.2166,...,3.8179,4.0115,3.9849,3.953,3.918,3.8789,3.8364,3.7908,3.7413,3.6885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,2023-01-09 00:00:00,3.9578,2.8257,2.7406,2.7608,2.8235,2.8581,2.8803,2.9556,3.0347,...,2.5667,2.7745,2.7049,2.6725,2.6423,2.6098,2.5728,2.533,2.4907,2.4438
505,2023-01-06 00:00:00,4.012,2.8551,2.7519,2.7691,2.8415,2.8821,2.9151,2.975,3.0578,...,2.5881,2.7969,2.7258,2.6939,2.664,2.6321,2.5951,2.5549,2.5119,2.4642
506,2023-01-05 00:00:00,4.282,3.1236,2.9789,2.9546,2.9806,3.0029,3.0167,3.0717,3.1308,...,2.6541,2.9041,2.8223,2.782,2.7455,2.7089,2.6695,2.6277,2.584,2.5361
507,2023-01-04 00:00:00,4.1432,3.0006,2.928,2.9487,2.9904,3.0121,3.0239,3.0862,3.1656,...,2.6813,2.9105,2.8366,2.8019,2.7686,2.733,2.6936,2.6524,2.6093,2.5616


In [5]:
def SwapsData(datapath, surfacespath):
    """
    function to read the pre-processed Swaptions implied vol data
    """

    # SETUP RETURNS ON UNDERLYING 
    returns = pd.read_excel(datapath, skiprows = 2).set_index("Ticker").sort_index().diff().dropna()
    
    # GET DATES
    dates_dt = returns.index
    
    # PROCESS VOLATILITY SURFACE DATA
    volatilities = pd.read_excel(surfacespath, skiprows=2).set_index("Ticker").sort_index()
    mat_n_ten = Inputs.maturity_tenor(surfacespath).T  

    # Initialize matrix
    surfaces_transform = np.empty((returns.shape[0], returns.shape[1]))  # Preallocate memory

    # Process dates
    for i, date in enumerate(dates_dt):
        d1 = pd.DataFrame(volatilities.loc[date])
        df = d1.join(mat_n_ten)  # Join precomputed maturity/tenor
        df.columns = ["Values", "Tenor", "Maturity"]
        surfaces_transform[i, :] = df["Values"].values  # Directly assign to matrix

    tenor = np.array([1/12, 1/4, 1/2, 3/4, 1, 1.5, 2, 3, 4, 5, 7, 10, 15, 20, 25])
    # \tau \in {1, 2, 3, 4, 5, 6, 7, 8, 9}
    tau = np.linspace(1, 9, 9) 
    
    taus, tenors = np.meshgrid(tau,tenor)
    
    return surfaces_transform, returns, tenor, tau, tenors, taus, dates_dt


In [6]:
SwapsData(DATA_PATH, SURFACE_PATH)

(array([[111.6, 136.4, 134.8, ...,  55.3,  54.3,  53.3],
        [108.2, 132.2, 130.8, ...,  55. ,  54. ,  53. ],
        [105.7, 129.8, 126.3, ...,  54.7,  53.6,  52.6],
        ...,
        [ 80.9, 101.7, 102.9, ...,  69.2,  68. ,  66.5],
        [ 79.9, 101.1, 102.2, ...,  69.1,  67.9,  66.4],
        [ 72.2,  96.8,  97.7, ...,  69.8,  68.6,  67.1]]),
             S0042FS 1Y1M BLC Curncy  S0042FS 2Y1M BLC Curncy  \
 Ticker                                                         
 2023-01-04                  -0.0236                  -0.0709   
 2023-01-05                   0.1388                   0.1230   
 2023-01-06                  -0.2700                  -0.2685   
 2023-01-09                  -0.0542                  -0.0294   
 2023-01-10                   0.0224                   0.0538   
 ...                             ...                      ...   
 2025-01-06                   0.0101                   0.0471   
 2025-01-07                   0.0171                   0.0

In [7]:
def DataPreprocesssing(datapath, surfacepath, vol_model='normal'):
    """
    function for preparing the data for VolGAN
    later to be split into train, val, test
    """
    surfaces_transform, returns, tenor, tau, tenors, taus, dates_dt = SwapsData(datapath,surfacepath)

    # PRODUCE REALIZED VOLATILITIES
    # Our returns are across different tenors and maturities (as if we have more than one underlying asset compared to just SPX)
    # This means that the realized volatilities we produce will actually be a matrix 

    #Realised volatility at time t-1
    
    realised_vol_tm1 = np.zeros((returns.shape[0]-22, returns.shape[1]))

    for i in range(len(realised_vol_tm1)):
        realised_vol_tm1[i] = np.sqrt(252/21) * np.sqrt((returns.iloc[i:(i+21)]**2).sum())

    # COLUMN INDEX is a specific asset with some tenor and maturity 
    # ROW INDEX is the date / time series of realised vol of that asset
    # This has shape (434, 144)

    #shift the time
    dates_t = dates_dt[22:]
    
    # SEPARATE OUR RETURNS
    #Return increment at t, t-1, t-2
    return_t = returns.iloc[22:].values
    return_tm1 = 252 * returns.iloc[21:-1].values
    return_tm2 = 252 * returns.iloc[20:-2].values
    # COLUMN index is a specific asset with some tenor and maturity, row index is the date / time series of realised vol of that asset

    # GET IMPLIED VOLS
    #implied vol at t and t-1

    # Note about volatility: if we are using normal model, then we can use the implied vol directly
    # If we are using log-normal model, then we need to convert the implied vol to log implied vol
    if vol_model == 'normal':
        #implied vol at t
        iv_t = surfaces_transform[22:]
        #implied vol at t-1
        iv_tm1 = surfaces_transform[21:-1]
        iv_inc_t = iv_t - iv_tm1
    elif vol_model == 'log':
        log_iv_t = np.log(surfaces_transform[22:])
        log_iv_tm1 = np.log(surfaces_transform[21:-1])
        #we want to simulate the increment at time t (t - t-1)
        log_iv_inc_t = log_iv_t - log_iv_tm1
    
    # same here the time series dates are the rows, the COLUMN is a specific underlying asset with specific tenor and maturity 
    
    # SET UP NORMALIZATION PARAMETERS (I think this is used later in model training, it might be useful to keep)
    
    # calculate normalisation parameters in case it is needed
    # underlying returns
    m_ret = np.mean(return_t[0:100], axis=0)
    sigma_ret = np.std(return_t[0:100], axis=0)
    # shape (144,)

    #realised vol
    m_rv = np.mean(realised_vol_tm1[0:100], axis=0)
    sigma_rv = np.std(realised_vol_tm1[0:100], axis=0)
    # shape (144,)

    if vol_model == 'normal':

        # normal implied vol
        m_iv = np.mean(iv_t[0:100], axis=0)
        sigma_iv = np.std(iv_t[0:100], axis=0)

        # normal implied vol increment
        m_iv_inc = np.mean(iv_inc_t[0:100], axis=0)
        sigma_iv_inc = np.std(iv_inc_t[0:100], axis=0)
    
        m_in = np.concatenate((m_ret,m_ret,m_rv,m_iv))
        sigma_in = np.concatenate((sigma_ret,sigma_ret,sigma_rv,sigma_iv))

        m_out = np.concatenate((m_ret,m_iv_inc))
        sigma_out = np.concatenate((sigma_ret,sigma_iv_inc))

        condition = np.concatenate((np.expand_dims(return_tm1,axis=2),np.expand_dims(return_tm2,axis=2),np.expand_dims(realised_vol_tm1,axis=2),np.expand_dims(iv_t, axis=2)),axis=2)
        # shape (434, 144, 4)
        # each asset at each time has the condition vector

    elif vol_model == 'log':

        # log implied vol 
        m_liv = np.mean(log_iv_t[0:100], axis=0)
        sigma_liv = np.std(log_iv_t[0:100], axis=0)
        # shape (144,)

        #log implied vol increment
        m_liv_inc = np.mean(log_iv_inc_t[0:100], axis=0)
        sigma_liv_inc = np.std(log_iv_inc_t[0:100], axis=0)
        # shape (144,)

        m_in = np.concatenate((m_ret,m_ret,m_rv,m_liv))
        sigma_in = np.concatenate((sigma_ret,sigma_ret,sigma_rv,sigma_liv))

        m_out = np.concatenate((m_ret,m_liv_inc))
        sigma_out = np.concatenate((sigma_ret,sigma_liv_inc))
        
        condition = np.concatenate((np.expand_dims(return_tm1,axis=2),np.expand_dims(return_tm2,axis=2),np.expand_dims(realised_vol_tm1,axis=2),np.expand_dims(log_iv_t, axis=2)),axis=2)    
        # shape (434, 144, 4)
        # each asset at each time has the condition vector

    #true: what we are trying to predict, increments at time t
    return_t_annualized = 252 * return_t

    if vol_model == 'normal':
        true = np.concatenate((np.expand_dims(return_t_annualized,axis=2),np.expand_dims(iv_inc_t, axis=2)),axis=2)
    elif vol_model == 'log':
        true = np.concatenate((np.expand_dims(return_t_annualized,axis=2),np.expand_dims(log_iv_inc_t, axis=2)),axis=2)
    
    # shape (434, 144, 2)
    # each asset at each time has the predicted annualized return and (normal or log) implied vol increment

    return true, condition, m_in, sigma_in, m_out, sigma_out, dates_t,  tenor, tau, tenors, taus


In [8]:
DataPreprocesssing(DATA_PATH, SURFACE_PATH, vol_model='normal')

(array([[[ 53.7516,   8.6   ],
         [ 40.1688,  10.2   ],
         [ 34.02  ,   9.5   ],
         ...,
         [  1.5624,   0.5   ],
         [  1.1088,   0.5   ],
         [  0.9324,   0.4   ]],
 
        [[ -0.2772,  -1.4   ],
         [ -0.882 ,  -2.6   ],
         [  1.386 ,  -2.1   ],
         ...,
         [ 13.2552,   0.    ],
         [ 12.8268,   0.    ],
         [ 12.1968,   0.1   ]],
 
        [[-11.9196,  -2.    ],
         [-11.5668,  -2.6   ],
         [-11.6424,  -2.4   ],
         ...,
         [ -9.2736,  -0.2   ],
         [ -8.568 ,  -0.3   ],
         [ -7.9632,  -0.3   ]],
 
        ...,
 
        [[ -6.7284,  -2.6   ],
         [ -1.1088,  -2.7   ],
         [  2.5704,  -2.8   ],
         ...,
         [  6.1992,   0.1   ],
         [  6.3   ,   0.    ],
         [  6.2496,   0.    ]],
 
        [[ -5.4432,  -1.    ],
         [ -4.032 ,  -0.6   ],
         [ -0.9324,  -0.7   ],
         ...,
         [ -2.268 ,  -0.1   ],
         [ -2.3436,  -0.1   ],
    

In [9]:
def DataTrainTest(datapath,surfacepath, tr, vol_model = 'normal', device = 'cpu'):
    """
    function to split the data into train, test
    tr are the proportions to use for testing
    tr is specifically the percentage of data to use for training
    """

    true, condition, m_in,sigma_in, m_out, sigma_out, dates_t,  tenor, tau, tenors, taus = DataPreprocesssing(datapath, surfacepath, vol_model=vol_model)

    data_tt = torch.from_numpy(m_in)
    m_in = data_tt.to(torch.float).to(device)
    data_tt = torch.from_numpy(m_out)
    m_out = data_tt.to(torch.float).to(device)
    data_tt = torch.from_numpy(sigma_in)
    sigma_in = data_tt.to(torch.float).to(device)
    data_tt = torch.from_numpy(sigma_out)
    sigma_out = data_tt.to(torch.float).to(device)

    n = true.shape[0]
    data_tt = torch.from_numpy(true)
    true_tensor = data_tt.to(torch.float).to(device)

    data_tt = torch.from_numpy(condition)
    condition_tensor = data_tt.to(torch.float).to(device)

    true_train = true_tensor[0:int(tr * n), :, :]
    true_test = true_tensor[int(tr * n):, :, :]

    condition_train = condition_tensor[0:int(tr * n), :, :]
    condition_test = condition_tensor[int(tr * n):, :, :]

    return true_train, true_test, condition_train,  condition_test,  m_in,sigma_in, m_out, sigma_out, dates_t,  tenor, tau, tenors, taus

In [10]:
DataTrainTest(DATA_PATH,SURFACE_PATH,.8)

(tensor([[[ 53.7516,   8.6000],
          [ 40.1688,  10.2000],
          [ 34.0200,   9.5000],
          ...,
          [  1.5624,   0.5000],
          [  1.1088,   0.5000],
          [  0.9324,   0.4000]],
 
         [[ -0.2772,  -1.4000],
          [ -0.8820,  -2.6000],
          [  1.3860,  -2.1000],
          ...,
          [ 13.2552,   0.0000],
          [ 12.8268,   0.0000],
          [ 12.1968,   0.1000]],
 
         [[-11.9196,  -2.0000],
          [-11.5668,  -2.6000],
          [-11.6424,  -2.4000],
          ...,
          [ -9.2736,  -0.2000],
          [ -8.5680,  -0.3000],
          [ -7.9632,  -0.3000]],
 
         ...,
 
         [[ -1.4616,  -1.1000],
          [  6.7536,  -2.0000],
          [  3.3264,  -1.1000],
          ...,
          [  2.9484,   0.0000],
          [  2.8728,   0.0000],
          [  2.7972,   0.0000]],
 
         [[  8.3916,  -3.1000],
          [  7.9884,  -1.2000],
          [  7.9632,   0.1000],
          ...,
          [ 11.1636,   0.2000],
 